In [ ]:
#| default_exp codetips

# Code Tips
> Pipeline for annotating code cells with notes

Code Tips operate the same way as Code Notes however they show up as tooltips instead and use the `tip` tag instead of the `explain`. Note that it also uses multiline CodeNotes by default so you need a start and an end. 

To use this make sure that you add in `hint.css` and `tips.css` to the `_quarto.yml` under the `css` section!

In [ ]:
#| export
from nbdev.config import get_config
from nbdev.process import NBProcessor, extract_directives
from nbdev.processors import Processor, mk_cell
from nbdev.export import nb_export
from nbdev.doclinks import nbglob
from nbdev.sync import write_nb

from fastcore.script import call_parse
from fastcore.xtras import Path

import shlex
import requests
import re

In [ ]:
from fastcore.test import test_eq

In [ ]:
#| export
HINT_CSS_URL = "https://raw.githubusercontent.com/muellerzr/til/master/nbs/hint.css"

In [ ]:
#| export
TIPS_CSS = """.nogap {
    padding: 0px;
    border-radius: 0px !important;
    outline: 0px 0px;
    margin-bottom: 0px !important;
}

.code-with-filename .code-with-filename-file {
    margin: 0px;
}
div.sourceCode {
    border: 0px;
    margin: 0px;
}

pre > code.sourceCode {
    white-space: pre;
    text-decoration: none;
}

.nogap > div.sourceCode > pre.sourceCode {
    line-height: 0px;
    padding-bottom: 0px;
    padding-top: 0px;
    margin-bottom: 0px;
    margin-top: 0px;
}"""

In [ ]:
#| export
def download_tooltip_css():
    config = get_config()
    css_path = config.nbs_path/"hint.css"
    if not css_path.exists():
        response = requests.get(HTML_CSS_URL)
        css_path.write_bytes(response.content)
        (config.nbs_path/"codetips.css").write_text(TIPS_CSS)
        print(f'Added custom css files. Please add `codetips.css` and `hint.css` to `_quarto.yml` under `format -> html -> css`')

In [ ]:
#| export
def write_tooltip_directives(
    # The tooltip text
    explanation: str,
    # A list of css directives to modify the tooltip
    hint_directives:list = ["rounded", "medium", "right"],
    # An optional filename to be rendered 
    filename:str = None,
):
    "Creates a tooltip in style of `hint_directives` with content `explanation`"
    hint_directives = [f".hint--{hint} " for hint in hint_directives]
    hint_directives[-1] = hint_directives[-1].rstrip() # for formatting
    tooltip = f'''#| classes: .nogap {"".join(hint_directives)}
#| aria-label: "{explanation}"'''
    if filename is not None:
        tooltip += f'\n#| filename: "{filename}"'
    return tooltip + "\n"

In [ ]:
test_eq(write_tooltip_directives("Some content"), '''#| classes: .nogap .hint--rounded .hint--medium .hint--right
#| aria-label: "Some content"
''')

In [ ]:
test_eq(write_tooltip_directives("Some content", filename="my/file/name"), '''#| classes: .nogap .hint--rounded .hint--medium .hint--right
#| aria-label: "Some content"
#| filename: "my/file/name"
''')

In [ ]:
#| export
def convert_explanation(explanation_cell, source):
    "Takes an explanation and source code and linkes them together in a new cell"
    filename = explanation_cell.directives_.pop("filename", None)
    explanation = re.sub(r'\*#|.*[\n]', "", explanation_cell.source)
    content = write_tooltip_directives(explanation, filename=filename)
    content += source
    return mk_cell(content, cell_type="code")

In [ ]:
#| export
def extract_code(start_code, end_code, source, instance_num, end_instance_num=0):
    "Finds code between start and finish potentially with instances to check"
    start_match = list(re.finditer(f'[ \t]*{start_code}', source))[int(instance_num)]
    start_char = start_match.span()[0]
    end_match = list(re.finditer(f'[ \t]*{end_code}', source))[int(end_instance_num)]
    end_char = end_match.span()[1]
    return source[start_char:end_char]

In [ ]:
#| export
def parse_code(code_cell, markdown_cell):
    "Parses directives to extract the code needed to be highlighted"
    directives = markdown_cell.directives_["tip"]
    directives = shlex.split(" ".join(directives))
    if len(directives) == 4:
        start_code, start_instance_num, end_code, end_instance_num = directives
    else:
        (start_code, start_instance_num, end_code), (end_instance_num) = directives, 0
    start_code, end_code = re.escape(start_code), re.escape(end_code)
    return extract_code(start_code, end_code, code_cell.source, start_instance_num, end_instance_num)

In [ ]:
#| export
class TipExportProc(Processor):
    "A proc that checks and reorganizes cells for documentation for proper explainations"
    offset = 0
    steps = []
    _i = 0
    def begin(self):
        self.reset()
        self.has_reset = False
        self.iter = 0
        self.offset = 0
    
    def reset(self):
        self.results = [mk_cell("::: {layout-ncol=1}", cell_type="markdown")]    
        self.code = []
        self._code = None
        self.found_explanation = False
        self.end_link = False
        self.explanations = []
        self.start_idx = None
        self.end_idx = None
    
    def cell(self, cell):
        if cell.cell_type == "code":
            if not self.found_explanation:
                self._code = cell
                self.start_idx = cell.idx_
                
        if cell.cell_type == "markdown" and "tip" in cell.directives_:
            self.found_explanation = True
            self.explanations.append(cell)
            
        if self.found_explanation:
            idx = cell.idx_ + 1
            if (len(self.nb.cells) <= idx+1) or ("tip" not in self.nb.cells[idx].directives_):
                self.end_link = True
                self.end_idx = cell.idx_ + 1
        
        if self.found_explanation and self.end_link:
            # Assume we have all code + explainations
            explanations = [self._code]
            for i,explanation in enumerate(self.explanations):
                source = parse_code(self._code, explanation)
                converted_explanation = convert_explanation(explanation, source)
                self.results.append(converted_explanation)
                self.nb.cells.remove(explanation)
            self.results.append(mk_cell(":::", cell_type="markdown"))
            self.nb.cells.remove(self._code)
            self.offset = 0
            for result in self.results:
                result.idx_ = self.nb.cells[self.start_idx - 1].idx_ + 1
                self.nb.cells.insert(self.start_idx + self.offset, result)
                self.offset += 1
            self.iter += 1
            self.reset()
            self.has_reset = True
            
            self.offset = 0
            for i,c in enumerate(self.nb.cells): c.idx_ = i

In [ ]:
#| export
@call_parse
def parse_notes():
    "Exports notebooks to parsed notes for documentation. Should be called in the workflow, not yourself!"
    for nb in nbglob(get_config().nbs_path):
        processor = NBProcessor(nb, [NoteExportProc], rm_directives=False)
        processor.process()
        write_nb(processor.nb, nb)

In [ ]:
processor = NBProcessor("test.ipynb", [TipExportProc], rm_directives=False)
processor.process()

In [ ]:
write_nb(processor.nb, "result.ipynb")

To use this, add `nbdev-extensions.codetips:TipExportProc` to your `settings.ini` and add the required css files to your `_quarto.yml`

Now it will automatically build your docs like notes!

In [ ]:
def addition(a,b):
    "Adds two numbers together"
    return a+b

#| tip a+b

We return the sum of `a` and `b`

#| tip return a+b

Return something

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()